## Social Media Monitoring and Analysis

Welcome! This notebook is geared to give you starter examples of how to use ChatGPT for social media monitoring and analysis. Specifically we will look at three use cases:
- Sentiment Analysis
- Topic Extraction
- Classification / Categorization

For our purposes we will look at a series of tweets for Sam's club and do our analysis. I'll walk you through all the basic steps needed to do the analysis. 

NOTE: To make it easier to focus on the process of analysis rather than getting the data, I've included a JSON file that has curated output from Twitter (now called, X) in it. If you want to sign up for a developer account so you can get live data, go here: https://developer.twitter.com/